# Exploratory Data Analysis (EDA) Round 2

The purpose of this notebook is to do another pass at the fire risk data and see 
if we can't examine the entire thing easily, get a sense of how far back our dataset goes.
To do this, we are going to use the offset function in the query (now that we know how to use it properly)


In [1]:
from __future__ import division, print_function # always future proof for python3
import pandas as pd

In [2]:
# for simplicity we'll store the url in a string that we'll then insert a new offset into each round
query_url = 'https://data.sfgov.org/resource/wbb6-uh78.json?$order=close_dttm%20DESC&$offset={}&$limit=1000'

In [3]:
df = pd.read_json(query_url.format('0'))

In [4]:
# I'm curious, how many pages of data do we have in this dataset, how many records?
# we could have done this programmatically by just continuing to go through the records
# and at a certain point we will need to do that, but for now
# we cheated and looked at the url https://data.sfgov.org/Public-Safety/Fire-Incidents/wr8u-xric
# and it shows we have 403,988 rows of data, so 403 pages that we'd need to page through if we wanted to create one
# big database.
# Certainly something we will do but for now let's grab the very last page and see what we've got
df = pd.read_json(query_url.format('403000'))


In [5]:
df.head()

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,other_units,primary_situation,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode
0,-,86 - investigate,-,951 Eddy St.,2003-01-11T02:57:40.000,NaN,2003-01-11T03:01:06.000,NaN,NaN,NaN,...,0,733 - smoke detector activation/malfunction,429 - multifamily dwellings,05,NaN,NaN,5,11,3,94102
1,-,86 - investigate,-,30th St. / Dolores St.,2003-01-11T02:20:29.000,NaN,2003-01-11T02:25:32.000,NaN,NaN,NaN,...,0,"311 - medical assist, assist ems crew","960 - street, other",11,NaN,NaN,8,4,1,94110
2,-,86 - investigate,-,Market St. / Steuart St.,2003-01-11T02:08:45.000,NaN,2003-01-11T02:15:11.000,NaN,NaN,NaN,...,0,"711 - municipal alarm system, street box false","960 - street, other",13,NaN,NaN,3,4,1,94105
3,-,86 - investigate,-,10th St. / Market St.,2003-01-11T01:12:53.000,NaN,2003-01-11T01:15:22.000,NaN,NaN,NaN,...,0,"710 - malicious, mischievous false call, other",963 - street or road in commercial area,36,NaN,NaN,6,9,2,94103
4,-,11 - extinguish,-,373 Ellis St.,2003-01-10T23:33:30.000,NaN,2003-01-10T23:36:17.000,NaN,NaN,NaN,...,0,"100 - fire, other",429 - multifamily dwellings,03,NaN,NaN,6,34,10,94102


In [6]:
# let's take a look at what cleanup work we need to do. 
# Generally a field that is just labeled as an "object" is something we wish to clean up
# This code will eventually find its way into a pipeline and moved to src
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988 entries, 0 to 987
Data columns (total 62 columns):
action_taken_other                              988 non-null object
action_taken_primary                            988 non-null object
action_taken_secondary                          988 non-null object
address                                         988 non-null object
alarm_dttm                                      988 non-null object
area_of_fire_origin                             60 non-null object
arrival_dttm                                    988 non-null object
automatic_extinguishing_system_present          36 non-null object
automatic_extinguishing_sytem_failure_reason    36 non-null object
automatic_extinguishing_sytem_perfomance        36 non-null object
automatic_extinguishing_sytem_type              36 non-null object
battalion                                       988 non-null object
box                                             20 non-null float64
call_number     

In [7]:
# As you can see above in the second-to-last row, there are 10 float objects, 15 int objects, and 37 unspecified objects.
# We will want to fix that.
# first, what are the date values and all?

# df.describe(include='all')
# Uh-oh! The describe() function above failed with a type error,
# stating that there's an unhashable type of `dict` in our data. We'll need to find that and fix it
# before we can do a proper analysis

# let's look at a single record
df.iloc[0]

action_taken_other                                                                              -
action_taken_primary                                                             86 - investigate
action_taken_secondary                                                                          -
address                                                                              951 Eddy St.
alarm_dttm                                                                2003-01-11T02:57:40.000
area_of_fire_origin                                                                           NaN
arrival_dttm                                                              2003-01-11T03:01:06.000
automatic_extinguishing_system_present                                                        NaN
automatic_extinguishing_sytem_failure_reason                                                  NaN
automatic_extinguishing_sytem_perfomance                                                      NaN
automatic_extinguish

In [8]:
# even though there are so many columns that some are hidden, we can see above that location is a dict field.
# so we should either consider removing it, or doing an eval to get it into the database
df.iloc[0]['location']


{u'coordinates': [-122.42329, 37.78251], u'type': u'Point'}

In [9]:
# pretty simple, it's just a location type and then lat long. I'm not sure we need the type "Point" but let's store it for now
# just to see what we get
# doing a simple check on http://www.latlong.net/
# confirmed the general lat long for SF is 37.774929, -122.419416 respectively,
# so the data is most likely flipped, where longitude is first, and latitude the second value.
# let's use ast/eval to convert the location into separate columns

# we first should confirm that we are dealing with strings or dicts
type(df.iloc[0]['location'])

dict

In [10]:
#import ast  # TODO: move this to the very top with the other imports
temp_df = df.join(pd.DataFrame(df["location"].to_dict()).T)
# we'll do the slow approach to this for now, just as an example, but later consider a faster solution
#for index, row in df.iterrows():
temp_df.head()    

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode,coordinates,type
0,-,86 - investigate,-,951 Eddy St.,2003-01-11T02:57:40.000,NaN,2003-01-11T03:01:06.000,NaN,NaN,NaN,...,429 - multifamily dwellings,05,NaN,NaN,5,11,3,94102,"[-122.42329, 37.78251]",Point
1,-,86 - investigate,-,30th St. / Dolores St.,2003-01-11T02:20:29.000,NaN,2003-01-11T02:25:32.000,NaN,NaN,NaN,...,"960 - street, other",11,NaN,NaN,8,4,1,94110,"[-122.424233, 37.742237]",Point
2,-,86 - investigate,-,Market St. / Steuart St.,2003-01-11T02:08:45.000,NaN,2003-01-11T02:15:11.000,NaN,NaN,NaN,...,"960 - street, other",13,NaN,NaN,3,4,1,94105,"[-122.394751, 37.79448]",Point
3,-,86 - investigate,-,10th St. / Market St.,2003-01-11T01:12:53.000,NaN,2003-01-11T01:15:22.000,NaN,NaN,NaN,...,963 - street or road in commercial area,36,NaN,NaN,6,9,2,94103,"[-122.417505, 37.77654]",Point
4,-,11 - extinguish,-,373 Ellis St.,2003-01-10T23:33:30.000,NaN,2003-01-10T23:36:17.000,NaN,NaN,NaN,...,429 - multifamily dwellings,03,NaN,NaN,6,34,10,94102,"[-122.412295, 37.784858]",Point


In [11]:
temp_df.iloc[0]['coordinates']

[-122.42329, 37.78251]

In [12]:
# let's quickly check if the `type` is necessary, or if it only contains point and we should just delete it
temp_df.type.value_counts(dropna=False)

Point    775
NaN      213
Name: type, dtype: int64

In [13]:
# let's check if the null values are important? 
temp_df[temp_df.type.isnull()].tail()

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode,coordinates,type
942,-,"22 - rescue, remove from harm",-,573 26th Av.,2003-01-01T03:36:46.000,NaN,2003-01-01T03:40:54.000,NaN,NaN,NaN,...,"000 - property use, other",14,NaN,NaN,NaN,4,1,NaN,NaN,NaN
949,-,86 - investigate,-,Buchanan St. / Moulton St.,2003-01-01T03:10:40.000,"80 - vehicle area, other",2003-01-01T03:12:12.000,NaN,NaN,NaN,...,"962 - residential street, road or residential dr",16,NaN,NaN,NaN,4,1,NaN,NaN,NaN
958,-,33 - provide advanced life support (als),-,Coleridge St. / Eugenia Av.,2003-01-01T01:30:02.000,NaN,2003-01-01T01:37:10.000,NaN,NaN,NaN,...,"962 - residential street, road or residential dr",32,NaN,NaN,NaN,6,2,NaN,NaN,NaN
971,-,"10 - fire, other",-,2 Parker Av.,2003-01-01T00:43:53.000,-,2003-01-01T00:47:00.000,-,-,-,...,"960 - street, other",10,-,-,NaN,4,1,NaN,NaN,NaN
985,-,86 - investigate,-,33rd Av. / Noriega St.,2003-01-01T00:18:19.000,-,2003-01-01T00:21:15.000,-,-,-,...,"962 - residential street, road or residential dr",18,-,-,NaN,4,1,NaN,NaN,NaN


In [14]:
# Can we assume that if type is null, so is coordinates, in which case location may also be null?
temp_df[temp_df.type.isnull()].coordinates.value_counts(dropna=False)

NaN    213
Name: coordinates, dtype: int64

In [15]:
# OK, so the above info shows we can drop type, and we can also drop location
# we'll also overwrite the original column and delete the temp_df
df = temp_df.drop(['type','location'],axis=1)

In [16]:
del temp_df

In [17]:
# TODO: when we circle back and do our next notebook and import, we should consider using int32, float32 to save memory
# TODO: use a publicly available database to fill in zipcode and lat lon with a geo lookup


In [18]:
# now that we've taken a look, remember that this is an OLD database, so it may have been overzealous to delete columns
# or decide that type wasn't useful.
# let's quickly do a check for the most recent 3,000 columns and see what data we get there.
temp_df = pd.read_json(query_url.format('0'))

In [19]:
temp_df = temp_df.join(pd.DataFrame(temp_df["location"].to_dict()).T)

In [20]:
temp_df.iloc[-1]

action_taken_other                                                                        NaN
action_taken_primary                                                           86 investigate
action_taken_secondary                                                                    NaN
address                                                                  7th St/folsom Street
alarm_dttm                                                            2016-06-24T16:21:32.000
area_of_fire_origin                                                                       NaN
arrival_dttm                                                          2016-06-24T16:27:13.000
automatic_extinguishing_system_present                                                    NaN
automatic_extinguishing_sytem_perfomance                                                  NaN
automatic_extinguishing_sytem_type                                                        NaN
battalion                                                   

In [21]:
temp_df.type.value_counts(dropna=False)

Point    999
NaN        1
Name: type, dtype: int64

In [22]:
# it appears that the coordinates always seem to represent a point, or is null. 
# so for now let's drop it
# one of the challenges with this dataset is that there are a LOT of columns
# we should determine which ones to trim down.
# since we now have the first 1,000 rows of data in `df` and the most recent records in `temp_df` let's concat the two into a single dataframe
temp_df = temp_df.drop(['type','location'],axis=1)


In [23]:
df = pd.concat([temp_df, df])

In [24]:
df.head()

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,other_units,primary_situation,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode
0,NaN,86 investigate,NaN,105 Aptos Avenue,2016-07-10T21:50:58.000,NaN,2016-07-10T21:54:42.000,NaN,NaN,NaN,...,0,733 smoke detector activation due to malfunction,215 high school/junior high school/middle school,15,NaN,NaN,7,11,3,94127
1,NaN,86 investigate,NaN,8th St/bryant Street,2016-07-10T19:49:15.000,NaN,2016-07-10T19:52:06.000,NaN,NaN,NaN,...,0,"711 municipal alarm system, malicious false alarm",nnn none,29,NaN,NaN,6,9,2,94103
2,NaN,61 restore municipal services,NaN,2nd St/brannan Street,2016-07-10T19:44:29.000,NaN,2016-07-10T19:47:35.000,NaN,NaN,NaN,...,0,"711 municipal alarm system, malicious false alarm","960 street, other",8,NaN,NaN,6,9,2,94107
3,NaN,86 investigate,NaN,8400 Oceanview Trail,2016-07-10T19:24:41.000,NaN,2016-07-10T19:28:56.000,NaN,NaN,NaN,...,0,733 smoke detector activation due to malfunction,429 multifamily dwelling,33,NaN,NaN,7,11,3,94132
4,NaN,86 investigate,NaN,636 Velasco Av A,2016-07-10T18:56:31.000,NaN,2016-07-10T19:00:13.000,NaN,NaN,NaN,...,1,531 smoke or odor removal,419 1 or 2 family dwelling,43,NaN,NaN,10,29,8,94134


In [25]:
# NOW let's try describe()
# we now have a problem because we have a list, which we created when we made coordinates
# let's fix that
# remember, first value is the longitude, and the second value is latitude
# don't do this for null values
mask = df.coordinates.notnull()
df.loc[mask, 'long'] = df[mask]['coordinates'].apply(lambda x: x[0])

In [26]:
df.loc[mask, 'lat'] = df[mask]['coordinates'].apply(lambda x: x[1])

In [27]:
df.iloc[-1]

action_taken_other                                                                             -
action_taken_primary                                                            86 - investigate
action_taken_secondary                                                                         -
address                                                                Broadway St. / Taylor St.
alarm_dttm                                                               2003-01-01T00:02:13.000
area_of_fire_origin                                                                            -
arrival_dttm                                                             2003-01-01T00:06:13.000
automatic_extinguishing_system_present                                                         -
automatic_extinguishing_sytem_failure_reason                                                   -
automatic_extinguishing_sytem_perfomance                                                       -
automatic_extinguishing_sytem_

In [28]:
# now we can delete the coordinates column
df = df.drop(['coordinates'],axis=1)

In [29]:
# again, let's try to describe the data
df.describe(include='all')

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode,long,lat
count,996,1988,1095,1985,1988,108,1988,50,36,41,...,1985,1988,50,53,1760.000000,1988.000000,1988.000000,1760.000000,1774.000000,1774.000000
unique,7,75,24,1689,1988,36,1986,6,1,4,...,131,87,5,7,NaN,NaN,NaN,NaN,NaN,NaN
top,-,86 - investigate,-,1 Sf Intl Airport,2016-06-27T09:47:26.000,-,2003-01-03T11:40:20.000,-,-,-,...,429 multifamily dwelling,1,-,-,NaN,NaN,NaN,NaN,NaN,NaN
freq,987,698,963,10,1,19,2,23,36,35,...,218,108,22,20,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.793182,8.836519,2.391851,94113.381250,-122.424762,37.770250
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.740296,6.354400,2.132943,10.204425,0.026985,0.026594
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,94102.000000,-122.513984,37.616901
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.000000,4.000000,1.000000,94105.000000,-122.435977,37.759051
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.000000,9.000000,2.000000,94110.000000,-122.419111,37.778016
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.000000,11.000000,3.000000,94121.000000,-122.407486,37.786679


In [30]:
# yay! it worked! we are getting closer to clean data but a long way from it.
# one important thing we should do is look at the close_dttm since that is what we used to order the data
# but it appears our dates are not viewed as dates
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 987
Data columns (total 63 columns):
action_taken_other                              996 non-null object
action_taken_primary                            1988 non-null object
action_taken_secondary                          1095 non-null object
address                                         1985 non-null object
alarm_dttm                                      1988 non-null object
area_of_fire_origin                             108 non-null object
arrival_dttm                                    1988 non-null object
automatic_extinguishing_system_present          50 non-null object
automatic_extinguishing_sytem_failure_reason    36 non-null object
automatic_extinguishing_sytem_perfomance        41 non-null object
automatic_extinguishing_sytem_type              41 non-null object
battalion                                       1988 non-null object
box                                             1020 non-null object
call_nu

In [31]:
# fortunately there was a nice use of the dttm to identify what should be datetime objects
# let's use that to filter and then convert
for col in df.columns:
    if 'dttm' in col:
        print(col)

alarm_dttm
arrival_dttm
close_dttm


In [32]:
# so there are three.. hmm I thought there were more? Let's start with converting these and then go from there
df['alarm_dttm'] = pd.to_datetime?

In [ ]:
df['alarm_dttm'] = pd.to_datetime

In [33]:
df['alarm_dttm'] = pd.to_datetime(df['alarm_dttm'])

In [34]:
df['arrival_dttm'] = pd.to_datetime(df['arrival_dttm'])
df['close_dttm'] = pd.to_datetime(df['close_dttm'])


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 987
Data columns (total 63 columns):
action_taken_other                              996 non-null object
action_taken_primary                            1988 non-null object
action_taken_secondary                          1095 non-null object
address                                         1985 non-null object
alarm_dttm                                      1988 non-null datetime64[ns]
area_of_fire_origin                             108 non-null object
arrival_dttm                                    1988 non-null datetime64[ns]
automatic_extinguishing_system_present          50 non-null object
automatic_extinguishing_sytem_failure_reason    36 non-null object
automatic_extinguishing_sytem_perfomance        41 non-null object
automatic_extinguishing_sytem_type              41 non-null object
battalion                                       1988 non-null object
box                                             1020 non-nul

In [36]:
# now that we are starting to clean up our data, it's time that we discard some of these unnecessary columns
# let's do a quick values count on each to see if we have any that are just blank field
for col in df.columns:
    print("\n", col.title(), "\n")
    print(df[col].value_counts(dropna=False), "\n")
    print("*"*20)
    


 Action_Taken_Other 

NaN                                               992
-                                                 987
34 transport person                                 2
63 restore fire alarm system                        2
86 investigate                                      2
62 restore sprinkler or fire protection system      1
20 - search & rescue, other                         1
84 refer to proper authority                        1
Name: action_taken_other, dtype: int64 

********************

 Action_Taken_Primary 

86 - investigate                                     698
86 investigate                                       556
11 - extinguish                                       79
11 extinguishment by fire service personnel           64
00 action taken, other                                51
63 restore fire alarm system                          50
70 assistance, other                                  33
63 - restore fire alarm system                        25
10

In [37]:
df.tail()

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_failure_reason,automatic_extinguishing_sytem_perfomance,...,property_use,station_area,structure_status,structure_type,supervisor_district,suppression_personnel,suppression_units,zipcode,long,lat
983,-,86 - investigate,-,291 10th St.,2003-01-01 00:19:50,-,2003-01-01 00:26:23,-,-,-,...,322 - alcohol/sub. abuse recovery center,36,-,-,6,11,3,94103,-122.412942,37.773001
984,-,32 - provide basic life support (bls),-,Market St. / Spear St.,2003-01-01 00:07:32,-,2003-01-01 00:07:46,-,-,-,...,419 - 1 or 2 family dwelling,13,-,-,6,0,0,94105,-122.395630,37.793790
985,-,86 - investigate,-,33rd Av. / Noriega St.,2003-01-01 00:18:19,-,2003-01-01 00:21:15,-,-,-,...,"962 - residential street, road or residential dr",18,-,-,NaN,4,1,NaN,NaN,NaN
986,-,87 - investigate - fire out on arrival,-,3rd St. / Harrison St.,2003-01-01 00:08:13,-,2003-01-01 00:11:19,-,-,-,...,"960 - street, other",08,-,-,6,4,1,94107,-122.397389,37.782554
987,-,86 - investigate,-,Broadway St. / Taylor St.,2003-01-01 00:02:13,-,2003-01-01 00:06:13,-,-,-,...,"962 - residential street, road or residential dr",02,-,-,3,4,1,94133,-122.413562,37.797038


In [38]:
# OK, so here's a list of columns that for the time being we'll just remove so that we can get a better sense of the 
# data and hopefully, more easily, grab the entire body of records, and not just these few rows

cols_to_drop = ["automatic_extinguishing_sytem_failure_reason",
                "automatic_extinguishing_sytem_type",
                "battalion",
                "box",
                "call_number",
                "detector_effectiveness",
                "detector_failure_reason",
                "ems_personnel",
                "ems_units",
                "exposure_number",
                "first_unit_on_scene",
                "ignition_factor_secondary",
                "mutual_aid",
                "no_flame_spead",
                "other_personnel",
                "other_units",
                "station_area",
                "supervisor_district"]
df = df.drop(cols_to_drop, axis=1)


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 987
Data columns (total 45 columns):
action_taken_other                          996 non-null object
action_taken_primary                        1988 non-null object
action_taken_secondary                      1095 non-null object
address                                     1985 non-null object
alarm_dttm                                  1988 non-null datetime64[ns]
area_of_fire_origin                         108 non-null object
arrival_dttm                                1988 non-null datetime64[ns]
automatic_extinguishing_system_present      50 non-null object
automatic_extinguishing_sytem_perfomance    41 non-null object
city                                        1975 non-null object
civilian_fatalities                         1988 non-null int64
civilian_injuries                           1988 non-null int64
close_dttm                                  1988 non-null datetime64[ns]
detector_alerted_occupants       

In [40]:
df.head()

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_perfomance,city,...,number_of_sprinkler_heads_operating,primary_situation,property_use,structure_status,structure_type,suppression_personnel,suppression_units,zipcode,long,lat
0,NaN,86 investigate,NaN,105 Aptos Avenue,2016-07-10 21:50:58,NaN,2016-07-10 21:54:42,NaN,NaN,San Francisco,...,NaN,733 smoke detector activation due to malfunction,215 high school/junior high school/middle school,NaN,NaN,11,3,94127,-122.466270,37.728947
1,NaN,86 investigate,NaN,8th St/bryant Street,2016-07-10 19:49:15,NaN,2016-07-10 19:52:06,NaN,NaN,San Francisco,...,NaN,"711 municipal alarm system, malicious false alarm",nnn none,NaN,NaN,9,2,94103,-122.406971,37.772527
2,NaN,61 restore municipal services,NaN,2nd St/brannan Street,2016-07-10 19:44:29,NaN,2016-07-10 19:47:35,NaN,NaN,San Francisco,...,NaN,"711 municipal alarm system, malicious false alarm","960 street, other",NaN,NaN,9,2,94107,-122.392082,37.781846
3,NaN,86 investigate,NaN,8400 Oceanview Trail,2016-07-10 19:24:41,NaN,2016-07-10 19:28:56,NaN,NaN,San Francisco,...,NaN,733 smoke detector activation due to malfunction,429 multifamily dwelling,NaN,NaN,11,3,94132,-122.467447,37.709976
4,NaN,86 investigate,NaN,636 Velasco Av A,2016-07-10 18:56:31,NaN,2016-07-10 19:00:13,NaN,NaN,San Francisco,...,NaN,531 smoke or odor removal,419 1 or 2 family dwelling,NaN,NaN,29,8,94134,-122.417959,37.709630


In [41]:
df[df.zipcode.isnull()].iloc[-1]

action_taken_other                                                                         -
action_taken_primary                                                        86 - investigate
action_taken_secondary                                                                     -
address                                                               33rd Av. / Noriega St.
alarm_dttm                                                               2003-01-01 00:18:19
area_of_fire_origin                                                                        -
arrival_dttm                                                             2003-01-01 00:21:15
automatic_extinguishing_system_present                                                     -
automatic_extinguishing_sytem_perfomance                                                   -
city                                                                                      SF
civilian_fatalities                                                   

In [42]:
# after googling around I found this API for doing a reverse geo lookup, using the lat long to get the nearest zipcode
# we will do that now, but try to be polite about it.
# http://api.geonames.org/findNearbyPostalCodesJSON?lat=37.728947&lng=-122.466270&username=demo

df[df.zipcode.isnull()].groupby(['lat','long'])['address'].value_counts(dropna=False)

lat        long         address              
37.616901  -122.384160  1 Sf Intl Airport        10
                        1                         1
37.812941  -122.477899  1019                      1
37.820633  -122.337061  800 80 Eb Bay Br Z Yb     1
37.866731  -122.432600  1 Angel Island Dr, Ai     1
dtype: int64

In [43]:
grouped = df[df.zipcode.isnull()].groupby(['lat','long'])


In [44]:
for name, group in grouped:
    print("")
    print(name)

    print(group['address'].value_counts(dropna=False))


(37.616900999999999, -122.38415999999999)
1 Sf Intl Airport    10
1                     1
Name: address, dtype: int64

(37.812941000000002, -122.47789899999999)
1019    1
Name: address, dtype: int64

(37.820633000000001, -122.33706100000001)
800 80 Eb Bay Br Z Yb    1
Name: address, dtype: int64

(37.866731000000001, -122.43259999999999)
1 Angel Island Dr, Ai    1
Name: address, dtype: int64


In [48]:
geo_url = "http://api.geonames.org/findNearbyPostalCodesJSON?lat={}&lng={}&username={}"
username = 'mikezawitkowski' # TODO: hide this in a config file not in source control

temp_df = pd.read_json(geo_url.format('37.616900999999999', '-122.38415999999999', username))
temp_df.head()

,postalCodes
0,"{u'distance': u'0', u'countryCode': u'US', u'p..."
1,"{u'distance': u'2.41637', u'countryCode': u'US..."
2,"{u'distance': u'3.97939', u'countryCode': u'US..."
3,"{u'distance': u'4.04594', u'countryCode': u'US..."
4,"{u'distance': u'4.68373', u'countryCode': u'US..."


In [51]:
temp_df.iloc[0]['postalCodes']

{u'adminCode1': u'CA',
 u'adminCode2': u'081',
 u'adminName1': u'California',
 u'adminName2': u'San Mateo',
 u'countryCode': u'US',
 u'distance': u'0',
 u'lat': 37.616901,
 u'lng': -122.38416,
 u'placeName': u'San Francisco',
 u'postalCode': u'94128'}

In [55]:
# OK, so let's populate the missing zipcodes
grouped = df[df.zipcode.isnull()].groupby(['lat','long'])

geo_url = "http://api.geonames.org/findNearbyPostalCodesJSON?lat={}&lng={}&username={}"
username = 'mikezawitkowski' # TODO: hide this in a config file not in source control

for name, group in grouped:
    lat, lon = name[0], name[1]
    print("lat: {}, long: {}".format(lat, lon))

    temp_df = pd.read_json(geo_url.format(lat, 
                                          lon, 
                                          username))
    mask = ((df.lat == float(lat)) & 
            (df['long'] == float(lon)) &
            (df.zipcode.isnull())
           )

    df.loc[mask, 'zipcode'] = temp_df.iloc[0]['postalCodes']['postalCode']


lat: 37.616901, long: -122.38416
lat: 37.812941, long: -122.477899
lat: 37.820633, long: -122.337061
lat: 37.866731, long: -122.4326


In [58]:
df[df.zipcode.isnull()].shape # there are still 214 values that are missing a lat lon AND zip

(214, 45)

In [67]:
mask = (df.zipcode.isnull())
geocolumns = ['address','city','neighborhood_district','zipcode','lat','long']
df[mask][geocolumns]

,address,city,neighborhood_district,zipcode,lat,long
515,908 Connecticut Street,San Francisco,NaN,NaN,NaN,NaN
8,1001 Larkin St.,SF,NaN,NaN,NaN,NaN
15,86 Rossi Av.,SF,NaN,NaN,NaN,NaN
18,6239 Geary Bl.,SF,NaN,NaN,NaN,NaN
29,Andover St. / Tompkins Av.,SF,NaN,NaN,NaN,NaN
32,Paris St. / Russia Av.,SF,NaN,NaN,NaN,NaN
34,Blanken Av. / Wheeler Av.,SF,NaN,NaN,NaN,NaN
35,1700 Steiner St.,SF,NaN,NaN,NaN,NaN
40,840 Van Ness Av.,SF,NaN,NaN,NaN,NaN
42,Eucalyptus Dr. / Melba Av.,SF,NaN,NaN,NaN,NaN


In [69]:
df[mask].iloc[0][geocolumns]

address                  908 Connecticut Street
city                              San Francisco
neighborhood_district                       NaN
zipcode                                     NaN
lat                                         NaN
long                                        NaN
Name: 515, dtype: object

In [73]:
# let's try building a street2coordinates endpoint
address = ('+').join(df[mask].iloc[0]['address'].split())
address = address + '%2c+San+Francisco%2c+CA'
s2c_url = "http://www.datasciencetoolkit.org/street2coordinates/"
temp_df = pd.read_json(s2c_url + address)
temp_df
# "http://www.datasciencetoolkit.org/street2coordinates/2543+Graystone+Place%2c+Simi+Valley%2c+CA+93065"

,"908 Connecticut Street, San Francisco, CA"
confidence,0.902
country_code,US
country_code3,USA
country_name,United States
fips_county,06075
latitude,37.7537
locality,San Francisco
longitude,-122.398
region,CA
street_address,908 Connecticut St


In [82]:
temp_df.loc['latitude'][0]

37.753653

In [74]:
# rename the data
df.loc[df.city == 'SF', 'city'] = 'San Francisco'

In [110]:
mask = (df.zipcode.isnull())
grouped = df[mask].groupby(['address','city'])

for name, group in grouped:
    if '/' in str(name[0]):  # the sign that it's an intersection, deal with those separately
        continue
    street, city = name[0], name[1]
    address = ('+').join(street.split())
    address = address + '%2c+' + '+'.join(city.split()) + '%2c+CA'
    s2c_url = "http://www.datasciencetoolkit.org/street2coordinates/"
    try:
        temp_df = pd.read_json(s2c_url + address)
    except ValueError as err:
        print(err)
        print(name)
        continue
    mask = ((df.address == street) & (df.city == city))
    df.loc[mask, 'lat'] = temp_df.loc['latitude'][0]
    df.loc[mask, 'long'] = temp_df.loc['longitude'][0]
    

If using all scalar values, you must pass an index
(u'319 Lower Fort Mason St.', u'FM')


In [112]:
df[df.zipcode.isnull() & (df.lat.isnull())]

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_perfomance,city,...,number_of_sprinkler_heads_operating,primary_situation,property_use,structure_status,structure_type,suppression_personnel,suppression_units,zipcode,long,lat
29,-,86 - investigate,-,Andover St. / Tompkins Av.,2003-01-10 16:42:53,NaN,2003-01-10 16:46:25,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,4,1,NaN,NaN,NaN
32,-,86 - investigate,-,Paris St. / Russia Av.,2003-01-10 16:05:35,NaN,2003-01-10 16:09:34,NaN,NaN,San Francisco,...,NaN,"700 - false alarm or false call, other","960 - street, other",NaN,NaN,4,1,NaN,NaN,NaN
34,-,86 - investigate,-,Blanken Av. / Wheeler Av.,2003-01-10 15:59:28,NaN,2003-01-10 16:02:29,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false",963 - street or road in commercial area,NaN,NaN,9,2,NaN,NaN,NaN
42,-,86 - investigate,-,Eucalyptus Dr. / Melba Av.,2003-01-10 14:46:03,NaN,2003-01-10 14:49:23,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,9,2,NaN,NaN,NaN
46,-,86 - investigate,-,Bright St. / Sargent St.,2003-01-10 13:56:24,NaN,2003-01-10 13:59:24,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,9,2,NaN,NaN,NaN
73,-,86 - investigate,-,Schwerin St. / Visitacion Av.,2003-01-10 03:15:30,NaN,2003-01-10 03:19:30,NaN,NaN,San Francisco,...,NaN,"700 - false alarm or false call, other",215 - middle/junior or high school,NaN,NaN,30,9,NaN,NaN,NaN
75,-,86 - investigate,-,Schwerin St. / Visitacion Av.,2003-01-10 03:17:04,NaN,2003-01-10 03:23:15,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,4,1,NaN,NaN,NaN
103,-,86 - investigate,-,Brewster St. / Joy St.,2003-01-09 19:14:57,NaN,2003-01-09 19:20:31,NaN,NaN,San Francisco,...,NaN,"651 - smoke scare, odor of smoke","962 - residential street, road or residential dr",NaN,NaN,23,6,NaN,NaN,NaN
119,-,86 - investigate,-,Blanken Av. / Wheeler Av.,2003-01-09 15:47:17,NaN,2003-01-09 15:50:35,NaN,NaN,San Francisco,...,NaN,"100 - fire, other","000 - property use, other",NaN,NaN,4,1,NaN,NaN,NaN
128,-,86 - investigate,-,Corbett Av. / Iron Al.,2003-01-09 14:35:04,NaN,2003-01-09 14:39:27,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","960 - street, other",NaN,NaN,4,1,NaN,NaN,NaN


In [63]:
# now, we go back and we fill in the zipcodes using lat long
geo_url = "http://api.geonames.org/findNearbyPostalCodesJSON?lat={}&lng={}&username={}"


action_taken_other                                                                       NaN
action_taken_primary                                                          86 investigate
action_taken_secondary                                                                   NaN
address                                                                     105 Aptos Avenue
alarm_dttm                                                               2016-07-10 21:50:58
area_of_fire_origin                                                                      NaN
arrival_dttm                                                             2016-07-10 21:54:42
automatic_extinguishing_system_present                                                   NaN
automatic_extinguishing_sytem_perfomance                                                 NaN
city                                                                           San Francisco
civilian_fatalities                                                   

In [109]:
df[df.zipcode.isnull()]

,action_taken_other,action_taken_primary,action_taken_secondary,address,alarm_dttm,area_of_fire_origin,arrival_dttm,automatic_extinguishing_system_present,automatic_extinguishing_sytem_perfomance,city,...,number_of_sprinkler_heads_operating,primary_situation,property_use,structure_status,structure_type,suppression_personnel,suppression_units,zipcode,long,lat
515,NaN,"30 emergency medical services, other",NaN,908 Connecticut Street,2016-07-01 22:34:15,NaN,2016-07-01 22:39:20,NaN,NaN,San Francisco,...,NaN,"300 rescue, ems incident, other","960 street, other",NaN,NaN,10,3,NaN,NaN,NaN
8,-,86 - investigate,-,1001 Larkin St.,2003-01-10 23:49:00,NaN,2003-01-10 23:52:13,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","960 - street, other",NaN,NaN,9,2,NaN,-122.418271,37.787037
15,-,86 - investigate,-,86 Rossi Av.,2003-01-10 21:14:33,NaN,2003-01-10 21:19:05,NaN,NaN,San Francisco,...,NaN,"400 - hazardous condition, other",419 - 1 or 2 family dwelling,NaN,NaN,4,1,NaN,NaN,NaN
18,-,"10 - fire, other",-,6239 Geary Bl.,2003-01-10 20:09:19,NaN,2003-01-10 20:12:07,NaN,NaN,San Francisco,...,NaN,"113 - cooking fire, confined to container",429 - multifamily dwellings,NaN,NaN,20,5,NaN,NaN,NaN
29,-,86 - investigate,-,Andover St. / Tompkins Av.,2003-01-10 16:42:53,NaN,2003-01-10 16:46:25,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,4,1,NaN,NaN,NaN
32,-,86 - investigate,-,Paris St. / Russia Av.,2003-01-10 16:05:35,NaN,2003-01-10 16:09:34,NaN,NaN,San Francisco,...,NaN,"700 - false alarm or false call, other","960 - street, other",NaN,NaN,4,1,NaN,NaN,NaN
34,-,86 - investigate,-,Blanken Av. / Wheeler Av.,2003-01-10 15:59:28,NaN,2003-01-10 16:02:29,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false",963 - street or road in commercial area,NaN,NaN,9,2,NaN,NaN,NaN
35,-,86 - investigate,-,1700 Steiner St.,2003-01-10 15:44:31,NaN,2003-01-10 15:47:07,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,1,1,NaN,-122.434730,37.784943
40,-,86 - investigate,-,840 Van Ness Av.,2003-01-10 15:08:39,NaN,2003-01-10 15:11:09,NaN,NaN,San Francisco,...,NaN,"730 - system malfunction, other",429 - multifamily dwellings,NaN,NaN,11,3,NaN,NaN,NaN
42,-,86 - investigate,-,Eucalyptus Dr. / Melba Av.,2003-01-10 14:46:03,NaN,2003-01-10 14:49:23,NaN,NaN,San Francisco,...,NaN,"711 - municipal alarm system, street box false","962 - residential street, road or residential dr",NaN,NaN,9,2,NaN,NaN,NaN


In [113]:
# TODO NEXT
# it's clear from teh above that the steps being used for cleaning are working fine. 
# What's next is we need to refactor the above into a cleaning script
# and move that to src
# we also should figure out a more lightweight method of getting the 
# the credentials from any .env file, perhaps a simple .json is fine here
# the biggest problem is how to make this work on someone else's library. 
# Do we instruct one to download the dstk image?
# Is there a container that's lightweight that they can install?